In [2]:


from box import Configbox

ERROR: Could not find a version that satisfies the requirement box (from versions: none)
ERROR: No matching distribution found for box


ModuleNotFoundError: No module named 'box'